# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Medaghri Alaoui Abdelmoumen
### <p style="text-align: right;"> &#9989; moumenalaoui

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

- git add HW_4_Regression_and_SVM_STUDENT
- git commit -m "hw-04"
- git push

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [44]:
#Write your code here
import pandas as pd

df = pd.read_csv("aussie_weather_data.csv")
df.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [45]:
#Write your code here
lst = []

cities = df["Location"].unique()

for city in cities:
    lst.append(city)
    
lst

['NorfolkIsland',
 'SydneyAirport',
 'WaggaWagga',
 'MelbourneAirport',
 'Mildura',
 'Watsonia',
 'Brisbane',
 'Cairns',
 'Townsville',
 'MountGambier',
 'Nuriootpa',
 'PerthAirport',
 'Perth',
 'AliceSprings',
 'Darwin']

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [46]:
#Write your code here
Mildura = df[df["Location"] == "Mildura"]
Mildura = Mildura.drop(columns =['Location'])
Mildura

,Year,Month,DayOfMonth,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
10679,2009,1,1,13.8,27.4,0.0,9.6,12.6,247.5,56.0,...,51.0,24.0,1009.8,1008.5,6.0,1.0,19.2,26.0,0,0
10680,2009,1,3,10.6,28.6,0.0,9.2,13.6,337.5,30.0,...,39.0,18.0,1019.2,1016.0,0.0,1.0,17.0,27.1,0,0
10681,2009,1,4,13.2,34.5,0.0,8.8,13.5,112.5,26.0,...,24.0,9.0,1014.2,1012.1,2.0,2.0,24.1,32.8,0,0
10682,2009,1,5,16.5,37.3,0.0,10.4,13.4,202.5,37.0,...,37.0,7.0,1013.7,1010.8,1.0,1.0,23.7,35.6,0,0
10683,2009,1,6,15.7,39.2,0.0,13.4,13.4,180.0,37.0,...,34.0,6.0,1011.8,1007.7,0.0,0.0,23.3,37.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13268,2017,6,17,9.6,13.7,1.0,0.6,1.0,157.5,15.0,...,99.0,87.0,1026.9,1025.7,8.0,8.0,10.4,13.3,0,0
13269,2017,6,18,1.7,14.4,0.4,0.2,2.4,247.5,15.0,...,100.0,92.0,1031.1,1030.5,0.0,0.0,5.5,12.5,0,0
13270,2017,6,20,1.1,17.9,0.2,1.4,7.4,90.0,17.0,...,100.0,50.0,1029.4,1026.8,0.0,0.0,6.0,17.3,0,0
13271,2017,6,22,-0.5,16.7,0.0,1.4,7.0,45.0,24.0,...,99.0,51.0,1028.2,1024.4,7.0,1.0,3.8,16.1,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [47]:
#Write your code here
from sklearn.model_selection import train_test_split

prep = Mildura["RainTomorrow"]
Mildura = Mildura.drop(columns =['RainTomorrow'])

X = Mildura
y = prep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=150)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

I think that it makes more sense for us to use logistic regression because the prediction is categorical, there are only 2 outcomes to the model.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [48]:
#Write your code here
import statsmodels.api as sm

logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.179692
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2075
Model:                          Logit   Df Residuals:                     2052
Method:                           MLE   Df Model:                           22
Date:                Fri, 28 Apr 2023   Pseudo R-squ.:                  0.4780
Time:                        22:00:35   Log-Likelihood:                -372.86
converged:                       True   LL-Null:                       -714.30
Covariance Type:            nonrobust   LLR p-value:                3.164e-130
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Year              0.0265      0.010      2.546      0.011       0.006       0.047
Month             0.

The most statistically significant features based on p-values are : 
- Year
- Month
- Evaporation
- Sunshine
- WindGustDir
- WindGustSpeed
- WindDir9am
- WindDir3pm
- Humidity3pm
- Pressure9am
- Pressure3pm
- Cloud3pm

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [49]:
#Write your code here
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

print(result.summary())

y_pred = result.predict(X_test)
y_pred_labels = (y_pred >= 0.5).astype(int)

conf_matrix = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(conf_matrix)
print(classification_report(y_test, y_pred_labels))

Optimization terminated successfully.
         Current function value: 0.179692
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2075
Model:                          Logit   Df Residuals:                     2052
Method:                           MLE   Df Model:                           22
Date:                Fri, 28 Apr 2023   Pseudo R-squ.:                  0.4780
Time:                        22:00:38   Log-Likelihood:                -372.86
converged:                       True   LL-Null:                       -714.30
Covariance Type:            nonrobust   LLR p-value:                3.164e-130
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Year              0.0265      0.010      2.546      0.011       0.006       0.047
Month             0.

#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

We can see that the precision is much greater than the recall. This means that the model is better at predicting the positive cases when it thinks they are positive, but it may miss some positive cases. We're talking about predicting if it will rain or not in this case

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [40]:
#Write your code for calculating the frequency of rain here.
count = 0
total = len(Mildura["RainToday"])
for val in Mildura["RainToday"]:
    if val == 1:
        count += 1

proportion = count / total
print(proportion)

0.1125674633770239


About 90% of the time.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [41]:
#Write your code here
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'gamma': [0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear']
}


svm = SVC()


grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, n_jobs=-1)


grid_search.fit(X_train, y_train)


print("Best parameters: ", grid_search.best_params_)

Best parameters:  {'C': 10.0, 'gamma': 0.01, 'kernel': 'linear'}


Best parameters:  {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report


clf = SVC(C=0.1, gamma=0.01, kernel='linear').fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Confusion matrix:\n", cm)

print(classification_report(y_test, y_pred))

Accuracy:  0.9210019267822736
Precision:  0.7619047619047619
Recall:  0.5079365079365079
Confusion matrix:
 [[446  10]
 [ 31  32]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       456
           1       0.76      0.51      0.61        63

    accuracy                           0.92       519
   macro avg       0.85      0.74      0.78       519
weighted avg       0.91      0.92      0.91       519



## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


We can see that the logistic regression is more accurate since the prediction is categorical. However both models seems better than the baseline.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


For SEED = 42 :
- logistic regression model : 
    - precision : 62%
    - recall : 38%
    - accuracy : 90%
    
- SVM : 
    - precision : 74%
    - recall : 42%
    - accuracy : 92%
    
    
For SEED = 70 :
- logistic regression model : 
    - precision : 72%
    - recall : 46%
    - accuracy : 90%
    
- SVM : 
    - precision : 77%
    - recall : 47%
    - accuracy : 93%
    
For SEED = 150 :
- logistic regression model : 
    - precision : 78%
    - recall : 49%
    - accuracy : 92%
    
- SVM : 
    - precision : 76%
    - recall : 51%
    - accuracy : 92%

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

We can see that the variable that changes the most is pretty much the recall that always increases by a couple percentage unlike the accuracy that doesn't really increase that much while the precision is between both it increase but doesn't increase a lot too.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

Based off the results, we can conclude that the models have very very similar results almots identical. The SVM gives better results but those are very close to those of the logistic regression model.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University